In [1]:
%load_ext autoreload
%autoreload 2

In [23]:
import itertools
import tensorcircuit as tc
import jax.numpy as jnp
import argparse
import jax
from jax import random
import tensorflow as tf
import tqdm
import seaborn as sns
import numpy as np
import optax
import pandas as pd
import matplotlib.pyplot as plt
import time
import sys

sys.path.append(".")

from queso.utils.io import IO

In [55]:
n = 14
k = 1
seed = 0

key = random.PRNGKey(seed)
# backend = tc.set_backend("jax")
backend = tc.set_backend("tensorflow")

def circuit(params):
    c = tc.Circuit(n)
    phi = np.array([0.0])
    for i in range(k):
        for j in range(n):
            c.r(j, theta=params[3 * j, i], alpha=params[3 * j + 1, i], phi=params[3 * j + 2, i])

        for j in range(1, n, 2):
            c.cnot(j - 1, j)

        for j in range(2, n, 2):
            c.cnot(j - 1, j)

    # interaction
    for j in range(n):
        c.rz(j, theta=phi[0])
    return c

def forward(params):
    c = circuit(params)
    return c.probability()

In [52]:
params = random.uniform(key, ([3 * n, k])) if tc.backend.name == "jax" else tf.random.normal(shape=[3 * n, k], stddev=0.1)

%timeit forward(params)
foward_jit = tc.backend.jit(forward)
%timeit forward_jit(params)

76.5 ms ± 1.27 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.55 ms ± 97.1 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [53]:
reverse = tc.backend.jacrev(forward)
reverse_jit = tc.backend.jit(reverse)
# %timeit reverse(params)
%timeit reverse_jit(params)
# print(p.shape, dp.shape)

10.9 s ± 361 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [56]:
c = circuit(params)
rho = tc.quantum.reduced_density_matrix(c, cut=[0, 1, 2])

AttributeError: 'Circuit' object has no attribute 'shape'

In [28]:
print(tc.backend.name)

tensorflow
